In [100]:
import sys
import math
import numpy as np
sys.path.insert(0, '..')
from net_framework import *
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import random
import json

In [101]:
term_data = pd.read_csv('term.txt', sep="\t", header=None)
term_data.columns = ["#Lnum", "#snum", "#cnum", "Term Abbrev"]
term_data.head()

,#Lnum,#snum,#cnum,Term Abbrev
0,1,1,1,LB
1,1,1,2,LB
2,1,1,3,LE
3,1,1,4,WK
4,1,1,5,LF


In [102]:
cnum_data = pd.read_csv('cnum-vhcm-lab-new.txt', sep="\t")
locations = cnum_data[['#cnum']]
locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
locations['Normalized-b'] = (cnum_data['b*'] - cnum_data['b*'].mean())/(cnum_data['b*'] - cnum_data['b*'].mean()).std() * 1/2
display(locations.head(5))

<ipython-input-102-f811cbef4bec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
<ipython-input-102-f811cbef4bec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
<ipython-input-102-f811cbef4bec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,#cnum,Normalized-L,Normalized-a,Normalized-b
0,141,1.000000,-0.025675,-0.138822
1,274,0.876301,-0.025504,-0.138822
2,129,0.876301,0.070445,-0.106039
3,230,0.876301,0.070101,-0.089951
4,302,0.876301,0.070617,-0.072041


In [103]:
locations = locations.sort_values('#cnum')
chip_num = list(locations['#cnum'])
lab_norm = [[row[2], row[3], row[4]] for row in locations.itertuples()]

In [104]:
l1 = term_data[term_data.get('#Lnum').eq(1)]
unique_symbols = list(l1['Term Abbrev'].unique())
l1_grouped = l1.groupby('#cnum')['Term Abbrev'].apply(list)
l1_chip_abbrev_percentage = [[(l1_grouped[i + 1].count(abbrev) / len(l1_grouped[i + 1])) for abbrev in unique_symbols] for i in range(len(l1_grouped))]

In [105]:
l1_result = pd.DataFrame(l1_chip_abbrev_percentage)
l1_result.index += 1
l1_result.index.name = '#cnum'
l1_result.columns = unique_symbols
print(l1_result)
chip_norm = []
#pull the percentage for each cnum
for x in chip_num:
    chip_norm.append((l1_result.loc[l1["#cnum"]==x]).values.tolist()[0])

         LB    LE    WK    LF     F     G     S   GB    FU
#cnum                                                     
1      0.36  0.00  0.00  0.04  0.08  0.52  0.00  0.0  0.00
2      0.12  0.00  0.04  0.12  0.60  0.04  0.00  0.0  0.08
3      0.00  0.96  0.04  0.00  0.00  0.00  0.00  0.0  0.00
4      0.28  0.00  0.40  0.00  0.08  0.04  0.20  0.0  0.00
5      0.04  0.00  0.00  0.20  0.68  0.08  0.00  0.0  0.00
...     ...   ...   ...   ...   ...   ...   ...  ...   ...
326    0.08  0.20  0.36  0.00  0.20  0.00  0.12  0.0  0.04
327    0.04  0.00  0.00  0.68  0.20  0.04  0.04  0.0  0.00
328    0.64  0.00  0.08  0.00  0.00  0.20  0.08  0.0  0.00
329    0.08  0.08  0.40  0.00  0.12  0.00  0.28  0.0  0.04
330    0.04  0.00  0.00  0.72  0.24  0.00  0.00  0.0  0.00

[330 rows x 9 columns]


In [106]:
node_num = range(10, 130, 10)
layer_num = range(1,6)
def mirror(arr, desired_length):
    if desired_length/len(arr) == 2:
        return arr + arr[::-1]
    else:
        return arr[:-1] + arr[::-1]

shape_collection = []
for node in node_num:
    for layer in layer_num:
        shape = []
        # it gets stuck on this particular one for some reason
        if layer == 1:
            shape = [node]
            shape_collection.append(shape)
        elif node//layer < 3:
            continue
        elif layer%2 == 0:
            # you are not leaving until you give me the right number
            while sum(shape) != node/2:
                shape = []
                # let the code randomly assign things that add up to what we want
                for l in range (0,int(layer/2)) :
                    shape.append(int(random.uniform(3,node+1)))
            
            shape = mirror(shape, layer)
            shape_collection.append(shape)
        else:
            while sum(shape) != node:
                shape = []
                # similar logic, but we let it mirror itself first
                for l in range (0, math.ceil(layer/2)) :
                    shape.append(random.randint(2,node))
                shape = mirror(shape, layer)    
            shape_collection.append(shape)

print(shape_collection)

[[10], [5, 5], [4, 2, 4], [20], [10, 10], [8, 4, 8], [7, 3, 3, 7], [6, 3, 2, 3, 6], [30], [15, 15], [2, 26, 2], [8, 7, 7, 8], [6, 3, 12, 3, 6], [40], [20, 20], [4, 32, 4], [6, 14, 14, 6], [10, 6, 8, 6, 10], [50], [25, 25], [6, 38, 6], [13, 12, 12, 13], [14, 8, 6, 8, 14], [60], [30, 30], [23, 14, 23], [9, 21, 21, 9], [24, 4, 4, 4, 24], [70], [35, 35], [32, 6, 32], [24, 11, 11, 24], [3, 21, 22, 21, 3], [80], [40, 40], [10, 60, 10], [31, 9, 9, 31], [32, 4, 8, 4, 32], [90], [45, 45], [15, 60, 15], [9, 36, 36, 9], [8, 8, 58, 8, 8], [100], [50, 50], [44, 12, 44], [43, 7, 7, 43], [41, 8, 2, 8, 41], [110], [55, 55], [25, 60, 25], [3, 52, 52, 3], [2, 16, 74, 16, 2], [120], [60, 60], [43, 34, 43], [36, 24, 24, 36], [23, 19, 36, 19, 23]]


Originally designed to find every single permutation
Oops misread directions


def check(arr):
    for a in arr:
        if a < 3:
            return False
    inverse = arr[::-1]
    if arr == inverse:
        return True
    else:
        return False

shape_collection = []

def trickle_down(node, layer):
    if layer > 1:
        for i in range(0, node[-1]):
            new_arr = node[:-1] + [node[-1]-i] + [i]
            trickle_down(new_arr,layer-1)
    else:
        if check(node):
            global shape_collection
            shape_collection.append(node)

for node in node_num:
    for layer in layer_num:
        trickle_down([node], layer)


In [107]:
#Number of training iterations
num_iters = 500

#Listing out the shapes of each model
colors_num = len(chip_norm[0])
input_size = 3

network_shapes = []
for s in shape_collection:
    network_shapes.append((input_size,s,colors_num))

#Learning rate of the network
rate = 0.001

#Generating Training Data
input_train, output_train, input_test, output_test = 0, 0, 0, 0
def shuffle():
    lab_train, lab_test, chip_train, chip_test = train_test_split(lab_norm, chip_norm, test_size=0.2, random_state = 3, shuffle = True)
    #test run on whole dataset
    #lab_train, lab_test = lab_norm, lab_norm
    #chip_train, chip_test = chip_norm, chip_norm
    
    global input_train, output_train, input_test, output_test
    input_train = torch.FloatTensor(lab_train)
    output_train = torch.FloatTensor(chip_train)
    input_test= torch.FloatTensor(lab_test)
    output_test = torch.FloatTensor(chip_test)

print(network_shapes)

[(3, [10], 9), (3, [5, 5], 9), (3, [4, 2, 4], 9), (3, [20], 9), (3, [10, 10], 9), (3, [8, 4, 8], 9), (3, [7, 3, 3, 7], 9), (3, [6, 3, 2, 3, 6], 9), (3, [30], 9), (3, [15, 15], 9), (3, [2, 26, 2], 9), (3, [8, 7, 7, 8], 9), (3, [6, 3, 12, 3, 6], 9), (3, [40], 9), (3, [20, 20], 9), (3, [4, 32, 4], 9), (3, [6, 14, 14, 6], 9), (3, [10, 6, 8, 6, 10], 9), (3, [50], 9), (3, [25, 25], 9), (3, [6, 38, 6], 9), (3, [13, 12, 12, 13], 9), (3, [14, 8, 6, 8, 14], 9), (3, [60], 9), (3, [30, 30], 9), (3, [23, 14, 23], 9), (3, [9, 21, 21, 9], 9), (3, [24, 4, 4, 4, 24], 9), (3, [70], 9), (3, [35, 35], 9), (3, [32, 6, 32], 9), (3, [24, 11, 11, 24], 9), (3, [3, 21, 22, 21, 3], 9), (3, [80], 9), (3, [40, 40], 9), (3, [10, 60, 10], 9), (3, [31, 9, 9, 31], 9), (3, [32, 4, 8, 4, 32], 9), (3, [90], 9), (3, [45, 45], 9), (3, [15, 60, 15], 9), (3, [9, 36, 36, 9], 9), (3, [8, 8, 58, 8, 8], 9), (3, [100], 9), (3, [50, 50], 9), (3, [44, 12, 44], 9), (3, [43, 7, 7, 43], 9), (3, [41, 8, 2, 8, 41], 9), (3, [110], 9), (3

In [ ]:
#Array of losses over training period for each network
output_file = {}
for n in node_num:
    output_file[n] = {}
    
for net_num, shape in enumerate(network_shapes):
    shuffle()
    print("Training: ",shape)
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    error_arr = []
    prev_error = 0
    strike = 0
    
    for i in range(num_iters):       
        NN.train(input_train, output_train)
        validation_error = NN.l1error(output_test, NN(input_test))
        #zero mistake counter at new training
        if i == 0:
            strike = 0
        #adding error to array
        error_arr.append(validation_error)
        #wait for them to grow up
        if prev_error < validation_error and i > 100:
            if strike > 3:
                print("Complete at iteration ", i, "\nFinal error: ", validation_error, "\n")
                break
            else:
                strike += 1
        prev_error = validation_error

    #Saves the training results in a filed named "Net 0", "Net 1", etc. 
    NN.saveWeights(model = NN, path = "saved_networks/Net " + str(net_num))
    output_file[sum(shape[1])][len(shape[1])] = error_arr

Training:  (3, [10], 9)
Complete at iteration  105 
Final error:  0.12613445520401 

Training:  (3, [5, 5], 9)
Complete at iteration  105 
Final error:  0.12572301924228668 

Training:  (3, [4, 2, 4], 9)
Complete at iteration  144 
Final error:  0.12210437655448914 

Training:  (3, [20], 9)
Complete at iteration  109 
Final error:  0.11713770031929016 

Training:  (3, [10, 10], 9)
Complete at iteration  105 
Final error:  0.13037103414535522 

Training:  (3, [8, 4, 8], 9)
Complete at iteration  111 
Final error:  0.13116085529327393 

Training:  (3, [7, 3, 3, 7], 9)
Complete at iteration  112 
Final error:  0.12910564243793488 

Training:  (3, [6, 3, 2, 3, 6], 9)
Complete at iteration  105 
Final error:  0.12291418015956879 

Training:  (3, [30], 9)
Complete at iteration  116 
Final error:  0.13103528320789337 

Training:  (3, [15, 15], 9)
Complete at iteration  105 
Final error:  0.14439505338668823 

Training:  (3, [2, 26, 2], 9)
Complete at iteration  110 
Final error:  0.1136960908

In [ ]:
with open('validation_errors.json', 'w') as f:
    json.dump(output_file, f)